In [1]:
# pip install pygame

In [2]:
import pygame
import sys
import random
import math
# import tensorflow as tf 
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.preprocessing import StandardScaler

pygame 2.5.2 (SDL 2.28.3, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


# 1.6 Ideas
1. enhance or add to input neurons to support pursuit behavior
    1a each input is a vector of a given offset to the organisms angle - vector returns count of prey/preds on vector, mean distance, nearest, maybe keep angle of nearest prey
2. e.g., prey density by angle? 
3. Update process so that top k pred genomes are carried forward to next simulation
4. Create prey energy multiplier (e.g., higher energy gain for center of map)

Update for 1.2 - create energy variable, reproduction and movement costs energy, prey generate energy per turn, predators gain energy = to prey's energy

In [3]:
# Constants
WIDTH, HEIGHT = 700, 700
FPS = 25
MAX_LOOPS = 5000


PREY_COLOR = (0, 204, 102)
BG_COLOR = (255, 255, 255)

STARTING_PREDS = 100
STARTING_PREY = 100
MAX_PREY = 500 #1000
MAX_PREDS = 500 #1000

#setting speed to 1 for both
PREDATOR_SPEED = 5
PREY_SPEED = 5

#ENERGY PARAMS
PREY_STARTING_ENERGY = 1
PRED_STARTING_ENERGY = 20

#ENERGY VALUE REQUIRED TO REPRODUCE
PRED_REPRODUCTION_ENERGY_THRESHOLD  = 40
PREY_REPRODUCTION_ENERGY_THRESHOLD = 15
PREY_REPRODUCTION_AGE = 1 #PREY MUST BE THIS OLD TO REPRODUCE
MAX_E = 100

MOVEMENT_COST = .1
#PREY_ENERGY_GAIN = 5
#exploring the concept of having a finite amount of energy added to the sim per turn (divided equally among prey)
TOTAL_PREY_ENERGY = STARTING_PREY * 4

PREDATOR_SURVIVAL_TAX = .1 #every round you lose this much energy if you don't eat
PREDATOR_STARVE = 0

PREDATOR_HIGH_ENERGY_COLOR = (232, 48, 48)
PREDATOR_LOW_ENERGY_COLOR = (41, 64, 86)

PREDATOR_SIZE =7
PREY_SIZE = 3


DETECTION_DISTANCE = 75
MAX_CONSUMPTION_DISTANCE = 10 #YOU MUST BE THIS CLOSE TO EAT/BE EATEN


#CREATE MUTATION
MUTATION_PROBABILITY = 0.2
MUTATION_STRENGTH=.05

#FORWARD DETECTION ANGLE (IE ARC THAT THE PREDATOR CAN SEE FORWARD)
FWD_ARC = 90


# Create Entity Classes

In [4]:
# Entity class
class Entity:
    def __init__(self, x, y, color, speed, age = 0,generation=1):
        self.x = x
        self.y = y
        self.color = color
        self.speed = speed
        self.age = age
        self.heading = random.uniform(0, 2 * math.pi)
        self.generation = 1

    def move(self):
        # Move the entity within the bounds of the playing area
        new_x = self.x + self.speed * math.cos(self.heading)
        new_y = self.y + self.speed * math.sin(self.heading)
        self.x = max(0, min(WIDTH, new_x))
        self.y = max(0, min(HEIGHT, new_y))
        #reduce entity energy by distance traveled ie speed
        self.energy = self.energy - MOVEMENT_COST
       


# Predator class (subclass of Entity)
class Predator(Entity):
    def __init__(self, x, y, color, speed):
        super().__init__(x, y, color, speed)
        # self.food = PREDATOR_STARTING_FOOD 
        self.genome = generate_random_nn(4,3) #this bit randomly creates everyone's little brain on init, first argument is sensory input count, second is action/output count
        self.energy = PRED_STARTING_ENERGY
        self.reproduce_binary = 1
        self.color = generate_random_color()
        # if self.energy >= PRED_REPRODUCTION_ENERGY_THRESHOLD:
        #     self.color = PREDATOR_HIGH_ENERGY_COLOR
        # else:
        #     self.color = PREDATOR_LOW_ENERGY_COLOR

    def reproduce(self):
        # Predator reproduction rules
        #If the predator has enough energy, then reproduce
        if self.energy > PRED_REPRODUCTION_ENERGY_THRESHOLD:  # if the predator has a full stomach, reproduce and reset food to starting value
            self.energy = self.energy - PRED_REPRODUCTION_ENERGY_THRESHOLD
            offspring = Predator(self.x, self.y, self.color, self.speed)
            offspring.genome = self.genome.copy() 
            offspring.color = self.color
            offspring.generation = self.generation + 1
            if MUTATION_PROBABILITY >= np.random.rand():
                # print('mutate')
                offspring.genome=mutate_genome(offspring.genome)
                offspring.color = mutate_color(offspring.color)
            return offspring
        else:
            return None

    def draw(self,screen):
        #draw the circle
        pygame.draw.circle(screen, self.color, (int(self.x), int(self.y)), PREDATOR_SIZE)
        #create our angle
        angle_line_len = PREDATOR_SIZE * 3
        angle_line_end_x = self.x + angle_line_len * math.cos(self.heading)
        angle_line_end_y = self.y + angle_line_len * math.sin(self.heading)
        pygame.draw.line(screen, (0, 0, 0), (int(self.x), int(self.y)), (int(angle_line_end_x), int(angle_line_end_y)), 1)

    #update predator color based on how well fed they are
    # def update_color(self):
    #     e = self.energy
    #     #make sure the value is in range
    #     if e >= PRED_REPRODUCTION_ENERGY_THRESHOLD:
    #         self.color = PREDATOR_HIGH_ENERGY_COLOR
    #     else:
    #         self.color = PREDATOR_LOW_ENERGY_COLOR


# Prey class (subclass of Entity)
class Prey(Entity):
    def __init__(self, x, y, color, speed):
        super().__init__(x, y, color, speed)
        self.age = 0
        self.energy = PREY_STARTING_ENERGY
        self.genome = generate_random_nn(4,3) #this bit randomly creates everyone's little brain on init, first argument is sensory input count, second is action/output count


    def reproduce(self):
        # Prey reproduction rules
        if self.age >= PREY_REPRODUCTION_AGE and self.energy > PREY_REPRODUCTION_ENERGY_THRESHOLD:
            self.energy = self.energy - PREY_REPRODUCTION_ENERGY_THRESHOLD
            offspring = Prey(self.x, self.y, self.color, self.speed) 
            offspring.genome = self.genome.copy() 
            offspring.generation = self.generation + 1
            if MUTATION_PROBABILITY >= np.random.rand():
                # print('mutate')
                offspring.genome=mutate_genome(offspring.genome)
            return offspring
        else:

            return None

# Create Perception Functions

In [13]:
# i want to create a function that finds the nearest entity within a specified distance


def calculate_distance(entity1, entity2):
    # Your logic to calculate the distance between two entities
    return np.sqrt((entity1.x - entity2.x)**2 + (entity1.y - entity2.y)**2)

def calculate_angle(entity1,entity2):
    # Calculate the angle between entity1 and entity2 relative to the x-axis
    delta_x = entity2.x - entity1.x
    delta_y = entity2.y - entity1.y

    # Use arctangent to get the angle in radians
    angle_radians = math.atan2(delta_y, delta_x)

    # Convert radians to degrees
    angle_degrees = math.degrees(angle_radians)

    return angle_degrees


#create a function that returns the distance and angle of the closest entity within range

def find_nearest_entity(entity,all_entities):
    #my understanding is that we can save some processing by calling the global variable into this function so python doesn't have to search the global vars for each iteration of the loop
    detect_dist = DETECTION_DISTANCE
    #init some variables we'll use
    nearest_entity = None
    min_dist = float('inf')
    for other_entity in all_entities:
        #If the other entity isn't this one 
        if other_entity != entity:
            dist = calculate_distance(entity,other_entity)
            #update the distance is within the specified detection distance AND it is lower than any prior entities, store the dist and angle
            if dist < min_dist and dist <=  detect_dist:
                min_dist = dist
                nearest_entity = other_entity
    if nearest_entity:
        angle_to_nearest = calculate_angle(entity,nearest_entity)     
        if isinstance(nearest_entity, Prey):
            prey_binary = 1
        else:
            prey_binary = 2
        return min_dist, angle_to_nearest, prey_binary
    else:
        return 0, 0, 0


#create a function that returns the distance to nearest edge
def dist_to_edge(entity,map_width,map_height):
    #convert the angle to radians
    rads = math.radians(entity.heading)
    #calculate dist to east/west edge
    dist_to_horizontal_edge = min(entity.x/math.cos(rads),(map_width-entity.x)/math.cos(rads))
    #distance to north south edge
    dist_to_vertical_edge = min(entity.y/math.sin(rads),(map_height - entity.y)/math.sin(rads))
    #return the lesser of two weevels
    return min(dist_to_horizontal_edge,dist_to_vertical_edge)


def generate_random_color():
    col =()
    while True:
        col =  (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
        if col != (0,0,0) and col != PREY_COLOR:
            break 
    return col

def mutate_color(col):
    incr = 30
    r = min(max(col[0]+random.choice([-incr,incr]),0),255)
    g = min(max(col[1]+random.choice([-incr,incr]),0),255)
    b = min(max(col[2]+random.choice([-incr,incr]),0),255)       
    new_col = (r,g,b)
    return new_col

#test function - calculate prey within range and angle
def count_prey_forward(predator,prey_list):
    detect_dist = DETECTION_DISTANCE
    detect_arc = FWD_ARC
    prey_count = 0
    #I need to cycle through prey, calculate distance
    #if dist is in range, calculate angle
    #determine if each prey's angle is within 90 degrees of the preds angle
    #add them to the count and return the final count in that range
    for prey in prey_list:
        dist_to_prey = calculate_distance(predator,prey)
        if dist_to_prey < detect_dist:
            angle_to_prey = calculate_angle(predator,prey)
            # Calculate the angle difference between predator's angle and angle to prey
            angle_diff = (angle_to_prey - predator.angle + 180) % 360 - 180
            # Check if the prey is within the specified angle range
            if -angle_to_prey / 2 <= angle_diff <= detect_arc / 2:
                prey_count += 1
    
    return prey_count

# NN functions

In [6]:
#neural network functions

#function to create random network weights
def generate_random_nn(input_dim,output_dim):
    weights = {
        'input_hidden':np.random.randn(input_dim,32),
        'hidden_output':np.random.randn(32,output_dim)
    }
    return weights

def apply_nn(input_data,weights):
    hidden_layer_output = np.dot(input_data, weights['input_hidden'])
    hidden_layer_activation = sigmoid(hidden_layer_output)

    output_layer_output = np.dot(hidden_layer_activation, weights['hidden_output'])
    output = sigmoid(output_layer_output)

    return output

def sigmoid(x):
    return 1/(1+np.exp(-x))

def scale_output(input,range_min,range_max):
    #conver the first item to a binary for move yes no
    scaled_output = range_min + (range_max-range_min) * input
    return scaled_output

def mutate_genome(genome):
    # Modify each array in the dictionary by 0.01
    modification_value = MUTATION_STRENGTH

    for key, array in genome.items():
        genome[key] = array + np.random.randn(*array.shape)*modification_value

    return genome

def count_genomes(pop):
    #first let's get a list of hidden layers from our predators
    hidden_layers = [entity.genome['input_hidden'] for entity in pop]
    sums = [i.sum() for i in hidden_layers]
    diversity = len(np.unique(sums))
    return diversity

# Main Loop

In [7]:
loops=[]
pred_n =[]
prey_n = []
pred_diversity = []
prey_diversity = []
max_pred_generation = []


# Initialize Pygame
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Predator-Prey Simulation")
clock = pygame.time.Clock()
font = pygame.font.Font(None,24)

# Create initial population
predators = [Predator(random.randint(0, WIDTH), random.randint(0, HEIGHT), PREDATOR_HIGH_ENERGY_COLOR, PREDATOR_SPEED) for _ in range(STARTING_PREDS)]
preys = [Prey(random.randint(0, WIDTH), random.randint(0, HEIGHT), PREY_COLOR, PREY_SPEED) for _ in range(STARTING_PREY)]

# Simulation loop
#init counter
loop_counter = 0


#begin our game loop
while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()

    screen.fill(BG_COLOR)
    #create a simple log df
    loops.append(loop_counter)
    pred_n.append(len(predators))
    prey_n.append(len(preys))
    population = predators + preys
    #lets calculate our gene pool
    #first let's get a list of hidden layers from our predators
    pred_genomes = count_genomes(predators)
    pred_diversity.append(pred_genomes)
    prey_genomes = count_genomes(preys)
    prey_diversity.append(prey_genomes)

    # Move predators, run perception step the movement step
    for predator in predators:
        #perception
        near_dist, near_angle, is_prey = find_nearest_entity(predator,population)
        distance_to_edge = dist_to_edge(predator,WIDTH,HEIGHT)
        
        nn_input_array = np.array([near_dist, near_angle,is_prey, distance_to_edge])
        nn_output_array = apply_nn(nn_input_array,predator.genome)

        #now lets scale output
        #first output is whether or not we want to be able to reproduce
        # reproduction_binary = scale_output(nn_output_array[3],0,1)
        # predator.reproduce_binary = round(reproduction_binary)
        update_speed = scale_output(nn_output_array[0],0,PREDATOR_SPEED)
        #third is our change angle binary
        change_angle_binary = scale_output(nn_output_array[1],0,1)
        change_angle_binary = round(change_angle_binary)
        #final is our angle
        update_angle = scale_output(nn_output_array[2],0,2*math.pi)

        # if move_binary==1: 
        predator.speed = update_speed
        #if the nn says to change angle, do that before moving
        if change_angle_binary==1:
            predator.heading = update_angle
        predator.move()


        #pygame.draw.circle(screen, predator.color, (int(predator.x), int(predator.y)), PREDATOR_SIZE)
        predator.draw(screen)

        # note: when movement becomes optional, this age increment will need to be disconnected from the move loop
        predator.age +=1
        # predator.update_color()

    # Move preys
    for prey in preys:
         #perception
        near_dist, near_angle, is_prey = find_nearest_entity(prey,population)
        distance_to_edge = dist_to_edge(prey,WIDTH,HEIGHT)
        #create outputs
        nn_input_array = np.array([near_dist, near_angle,is_prey, distance_to_edge])
        nn_output_array = apply_nn(nn_input_array,prey.genome)
        update_speed = scale_output(nn_output_array[0],0,PREY_SPEED)
        change_angle_binary = scale_output(nn_output_array[1],0,1)
        change_angle_binary = round(change_angle_binary)
        #final is our angle
        update_angle = scale_output(nn_output_array[2],0,2*math.pi)
        prey.speed = update_speed
        #if the nn says to change angle, do that before moving
        if change_angle_binary==1:
            prey.heading = update_angle
        prey.move()
        pygame.draw.circle(screen, prey.color, (int(prey.x), int(prey.y)), PREY_SIZE)
        #age our prey
        prey.age +=1 
        #CALCULATE THE ENERGY GAIN PER PREY
        prey_energy_gain = TOTAL_PREY_ENERGY / len(preys)

        prey.energy = min(prey.energy + prey_energy_gain,MAX_E)
        
        #prey reproduces
        new_prey = prey.reproduce()
        if new_prey and len(preys) < MAX_PREY:
            preys.append(new_prey)

    # Check for collisions (predator catching prey)
    for predator in predators:
        for prey in preys:

            distance = math.sqrt((predator.x - prey.x)**2 + (predator.y - prey.y)**2)
            if distance < MAX_CONSUMPTION_DISTANCE:
                predator.energy = min(MAX_E,predator.energy + prey.energy)
                preys.remove(prey)

    # Now that the preds have eaten, let's see if they reproduce
    for predator in predators:
        #if they have starved, lets remove them
        predator.energy -= PREDATOR_SURVIVAL_TAX
        if predator.energy <= PREDATOR_STARVE:
            predators.remove(predator)
        else:
        #if not, give them a chance to reproduce
            new_predator = predator.reproduce()
            if new_predator and len(predators) < MAX_PREDS:
                predators.append(new_predator)

    #lets figure out what hte max geneartion of predators is
    for predator in predators:
        max_gen = 1
        max_gen = max(predator.generation,max_gen)

    max_pred_generation.append(max_gen)
        
   
    # Display the population counts
    text_surface = font.render(f'Pred n: {len(predators)}', True, (0, 0, 0))
    screen.blit(text_surface, (10, 10))

    text_surface = font.render(f'Prey n: {len(preys)}', True, (0, 0, 0))
    screen.blit(text_surface, (10, 40))

     # Display the loop counter
    text_surface = font.render(f'Loop: {loop_counter}', True, (0, 0, 0))
    screen.blit(text_surface, (10, 70))

     # Display predator diversity
    text_surface = font.render(f'Pred Gene Pool: {pred_genomes}', True, (0, 0, 0))
    screen.blit(text_surface, (10, 25))

    
     # Display predator diversity
    text_surface = font.render(f'Prey Gene Pool: {prey_genomes}', True, (0, 0, 0))
    screen.blit(text_surface, (10, 55))

        
     # Display predator generation
    text_surface = font.render(f'Mx Pred Gen: {max_gen}', True, (0, 0, 0))
    screen.blit(text_surface, (10, 85))



    loop_counter +=1
    pygame.display.flip()
    clock.tick(FPS)

  
    # Check if the number of predators or prey is zero
    if len(predators) == 0 or len(preys) == 0 or loop_counter == MAX_LOOPS:
        pygame.quit()
        sys.exit()




C:\Users\david\AppData\Local\Temp\ipykernel_25232\2953599813.py:21: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))


SystemExit: 

C:\Users\david\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\IPython\core\interactiveshell.py:3405: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [9]:
p = Predator(1,1,PREY_COLOR,1)

In [12]:
count_prey_forward(p,preys)

TypeError: count_prey_forward() missing 1 required positional argument: 'angle_range'

# After Action

In [ ]:

df = pd.DataFrame({'loop':loops,
                'pred_count':pred_n,
                'prey_count':prey_n,
                'pred_diversity':pred_diversity,
                'prey_diversity':prey_diversity})
scaler = StandardScaler()

df['pred_count_scaled'] = scaler.fit_transform(df[['pred_count']])
df['prey_count_scaled'] = scaler.fit_transform(df[['prey_count']])
df['pred_diversity_scaled'] = scaler.fit_transform(df[['pred_diversity']])
df['prey_diversity_scaled'] = scaler.fit_transform(df[['prey_diversity']])


df.head()
plt_df = df.melt(id_vars = 'loop',value_vars=['pred_count_scaled','prey_count_scaled'])
fig = px.line(plt_df,x='loop',y='value',color='variable',width = 1400,title = 'Time to Extinct '+ str(df.loop.max()),
              template = 'simple_white')
fig.update_layout(hovermode="x unified")
fig.show()
plt_df = df.melt(id_vars = 'loop',value_vars=['pred_diversity_scaled','prey_diversity_scaled'])
fig = px.line(plt_df,x='loop',y='value',color='variable',width = 1400,title = 'Time to Extinct '+ str(df.loop.max()),
              template = 'simple_white')
fig.update_layout(hovermode="x unified")
fig.show()

In [ ]:

df.head()
plt_df = df.melt(id_vars = 'loop',value_vars=['pred_count','prey_count','pred_diversity','prey_diversity'])
fig = px.line(plt_df,x='loop',y='value',color='variable',facet_row = 'variable',width = 1400,height=1400,title = 'Time to Extinct '+ str(df.loop.max()),
              template = 'simple_white')
fig.update_layout(hovermode="x unified")
fig.update_yaxes(matches=None)
fig.show()